# Projeto 2 - Ciência dos Dados

## Integrantes:
* Gabriela Kimi
* Luiza Ehrenberger
* Pedro Barão
* Rafael Paolino

## Introdução:

## Objetivo: fazer uso de métodos de regressão para prever o preço de carros, usando suas características como base.

Para isso, fizemos uso de uma base de dados que leva em conta as características de um carro e o preço sugerido pelo fabricante. Alguns atributos utilizados para estimar o preço são: 

* marca
* modelo
* ano de fabricação
* tipo de motor

<a href= " https://www.kaggle.com/CooperUnion/cardataset " > Link para a base de dados "Car Features and MSRP"</a>

### Importando as bibiotecas necessárias: 

In [1024]:
%matplotlib notebook

import pandas as pd
import numpy as np
from scipy.stats import norm, probplot
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from sklearn.tree import DecisionTreeRegressor 
from sklearn.preprocessing import OneHotEncoder
from IPython.display import display
import seaborn as sns

### Função de regressão linear:

#### Y: coluna do DataFrame utilizada como variável resposta. (TARGET)
#### X: coluna(s) do DataFrame utilizada(s) como variável(is) explicativas. (FEATURES)

In [1025]:
def regress(Y,X):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    
    return results

### DataFrame da base de dados

In [1026]:
data = pd.read_csv("data.csv")

In [1027]:
data.head(3)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350


## Mineirando Dados e Características do Dataset

### Colunas que serão utilizadas: 

In [1028]:
data=data.drop('Popularity',axis=1)
data.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'MSRP'],
      dtype='object')

### Descrevendo as variáveis que serão utilizadas:

* Make - Marca do carro 
* Modelo - Modelo do carro 
* Year - Ano de lançamento do carro 
* Engine Fuel Type - Tipo de combustível utilizado 
* Engine HP - Potência do motor (HP)
* Engine Cylinders - Número de cilindros do motor 
* Transmission Type - Tipo de transmissão (Câmbio)
* Driven wheels - Tipo de tração
* Number of doors - Número de portas 
* Market Category - Categorias usadas no mercado 
* Vehicle Size - Classificação do tamanho do veículo 
* Vehicle Style - Classificação de estilo do veículo
* Highway MPG - Média de rendimento de gasolina na auto-estrada (milhas por galão)
* City MPG - Média de rendimento de gasolina na cidade (milhas por galão)
* MSRP - preço de varejo recomendado pelo fabricante do carro (dólares)

## Remoção de células sem valores

In [1029]:
data=data.dropna()  
data.isnull().sum()

Make                 0
Model                0
Year                 0
Engine Fuel Type     0
Engine HP            0
Engine Cylinders     0
Transmission Type    0
Driven_Wheels        0
Number of Doors      0
Market Category      0
Vehicle Size         0
Vehicle Style        0
highway MPG          0
city mpg             0
MSRP                 0
dtype: int64

### Codificação de colunas categóricas para integer e hot encode:


In [1030]:
data=data.drop(['Make','Model','Vehicle Style','Market Category'],axis=1)
data_2=pd.get_dummies(data)

## Criação de DataFrames de treinamento e de teste:

In [1031]:
train = data.sample(7200)

In [1032]:
train= data_2.sample(frac=0.9)

test=data_2.drop(train.index)

In [1033]:
train.columns

Index(['Year', 'Engine HP', 'Engine Cylinders', 'Number of Doors',
       'highway MPG', 'city mpg', 'MSRP', 'Engine Fuel Type_diesel',
       'Engine Fuel Type_electric',
       'Engine Fuel Type_flex-fuel (premium unleaded recommended/E85)',
       'Engine Fuel Type_flex-fuel (premium unleaded required/E85)',
       'Engine Fuel Type_flex-fuel (unleaded/E85)',
       'Engine Fuel Type_premium unleaded (recommended)',
       'Engine Fuel Type_premium unleaded (required)',
       'Engine Fuel Type_regular unleaded',
       'Transmission Type_AUTOMATED_MANUAL', 'Transmission Type_AUTOMATIC',
       'Transmission Type_DIRECT_DRIVE', 'Transmission Type_MANUAL',
       'Transmission Type_UNKNOWN', 'Driven_Wheels_all wheel drive',
       'Driven_Wheels_four wheel drive', 'Driven_Wheels_front wheel drive',
       'Driven_Wheels_rear wheel drive', 'Vehicle Size_Compact',
       'Vehicle Size_Large', 'Vehicle Size_Midsize'],
      dtype='object')

### Plotagem de gráficos scatter

In [1034]:
#sns.pairplot(train_2)

## Modelos de Predição

### Explicar o que caea modelo de predição faz e explicar como funciona a biblioteca escolhida

### Regressão linear MMQ:

In [1035]:
X=train.drop(['MSRP'],axis=1)
Y=train[['MSRP']]

results=regress(Y,X)
results.summary()

D:\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MSRP   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                     347.8
Date:                Mon, 29 Nov 2021   Prob (F-statistic):               0.00
Time:                        18:15:06   Log-Likelihood:                -89031.
No. Observations:                7276   AIC:                         1.781e+05
Df Residuals:                    7253   BIC:                         1.783e+05
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
=================================================================================================================================
                                                                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------
const                                                         -1.254e+05   1.35e+05     -0.928      0.353    -3.9e+05     1.4e+05
Year                                                             45.8704    129.659      0.354      0.724    -208.300     300.040
Engine HP                                                       286.2689     12.530     22.846      0.000     261.706     310.832
Engine Cylinders                                               1.372e+04    691.877     19.828      0.000    1.24e+04    1.51e+04
Number of Doors                                                -502.5436    841.118     -0.597      0.550   -2151.380    1146.293
highway MPG                                                     -18.1230    138.289     -0.131      0.896    -289.211     252.965
city mpg                                                       1291.6108    183.808      7.027      0.000     931.293    1651.929
Engine Fuel Type_diesel                                         731.3560   1.87e+04      0.039      0.969   -3.59e+04    3.74e+04
Engine Fuel Type_electric                                     -5.474e+04   3.64e+04     -1.502      0.133   -1.26e+05    1.67e+04
Engine Fuel Type_flex-fuel (premium unleaded recommended/E85) -1.164e+04   2.06e+04     -0.564      0.573   -5.21e+04    2.88e+04
Engine Fuel Type_flex-fuel (premium unleaded required/E85)    -2157.7792   1.92e+04     -0.112      0.911   -3.98e+04    3.55e+04
Engine Fuel Type_flex-fuel (unleaded/E85)                     -2.423e+04   1.88e+04     -1.290      0.197   -6.11e+04    1.26e+04
Engine Fuel Type_premium unleaded (recommended)               -1.774e+04   1.85e+04     -0.959      0.337    -5.4e+04    1.85e+04
Engine Fuel Type_premium unleaded (required)                    453.5657   1.83e+04      0.025      0.980   -3.53e+04    3.63e+04
Engine Fuel Type_regular unleaded                              -1.61e+04   1.82e+04     -0.885      0.376   -5.17e+04    1.96e+04
Transmission Type_AUTOMATED_MANUAL                             6207.6604   2.86e+04      0.217      0.828   -4.99e+04    6.23e+04
Transmission Type_AUTOMATIC                                   -1.566e+04   2.88e+04     -0.544      0.586   -7.21e+04    4.07e+04
Transmission Type_DIRECT_DRIVE                                -2.146e+04   3.96e+04     -0.542      0.588   -9.91e+04    5.62e+04
Transmission Type_MANUAL                                      -2.407e+04   2.84e+04     -0.849      0.396   -7.97e+04    3.15e+04
Transmission Type_UNKNOWN                                     -7.044e+04   3.64e+04     -1.938      0.053   -1.42e+05     820.042
Driven_Wheels_all wheel drive                                 -2.406e+04   3.39e+04     -0.71

### Remoção de colunas com valor p > 10%:

In [1036]:
X=X.drop(['Year','Number of Doors','highway MPG','Engine Fuel Type_diesel','Engine Fuel Type_electric','Engine Fuel Type_electric','Engine Fuel Type_electric','Engine Fuel Type_flex-fuel (premium unleaded recommended/E85)','Transmission Type_AUTOMATIC','Engine Fuel Type_premium unleaded (required)','Engine Fuel Type_flex-fuel (premium unleaded required/E85)'],axis=1)
Y=train[['MSRP']]



## Regressão Linear

In [1037]:
results=regress(Y,X)
results.summary()

D:\Anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MSRP   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                     510.0
Date:                Mon, 29 Nov 2021   Prob (F-statistic):               0.00
Time:                        18:15:06   Log-Likelihood:                -89033.
No. Observations:                7276   AIC:                         1.781e+05
Df Residuals:                    7260   BIC:                         1.782e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
===================================================================================================================
                                                      coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
const                                           -7.796e+04   3362.676    -23.183      0.000   -8.45e+04   -7.14e+04
Engine HP                                         287.9734     10.493     27.443      0.000     267.403     308.544
Engine Cylinders                                 1.367e+04    643.471     21.241      0.000    1.24e+04    1.49e+04
city mpg                                         1251.8917    138.438      9.043      0.000     980.513    1523.270
Engine Fuel Type_flex-fuel (unleaded/E85)       -2.427e+04   2477.031     -9.799      0.000   -2.91e+04   -1.94e+04
Engine Fuel Type_premium unleaded (recommended) -1.788e+04   1920.965     -9.310      0.000   -2.17e+04   -1.41e+04
Engine Fuel Type_regular unleaded               -1.631e+04   1798.157     -9.071      0.000   -1.98e+04   -1.28e+04
Transmission Type_AUTOMATED_MANUAL               2.217e+04   2488.943      8.906      0.000    1.73e+04     2.7e+04
Transmission Type_DIRECT_DRIVE                  -5.139e+04   1.72e+04     -2.983      0.003   -8.52e+04   -1.76e+04
Transmission Type_MANUAL                        -8362.9368   1720.422     -4.861      0.000   -1.17e+04   -4990.409
Transmission Type_UNKNOWN                       -5.479e+04   2.89e+04     -1.896      0.058   -1.11e+05    1856.549
Driven_Wheels_all wheel drive                   -1.228e+04   1317.575     -9.322      0.000   -1.49e+04   -9700.255
Driven_Wheels_four wheel drive                  -3.073e+04   2052.749    -14.971      0.000   -3.48e+04   -2.67e+04
Driven_Wheels_front wheel drive                 -8178.3484   1522.507     -5.372      0.000   -1.12e+04   -5193.792
Driven_Wheels_rear wheel drive                  -2.676e+04   1405.744    -19.039      0.000   -2.95e+04    -2.4e+04
Vehicle Size_Compact                            -1.104e+04   1483.998     -7.437      0.000   -1.39e+04   -8127.331
Vehicle Size_Large                              -3.489e+04   1752.307    -19.909      0.000   -3.83e+04   -3.15e+04
Vehicle Size_Midsize                            -3.203e+04   1329.728    -24.091      0.000   -3.46e+04   -2.94e+04
==============================================================================
Omnibus:                    13919.664   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         42666557.464
Skew:                          14.616   Prob(JB):                         0.00
Kurtosis:                     377.008   Cond. No.                     2.62e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 9.48e-29. This might indicate that there a

## Regressão em árvore:

In [1038]:
X=test.drop(['MSRP','Year','Number of Doors','highway MPG','Engine Fuel Type_diesel','Engine Fuel Type_electric','Engine Fuel Type_electric','Engine Fuel Type_electric','Engine Fuel Type_flex-fuel (premium unleaded recommended/E85)','Transmission Type_AUTOMATIC','Engine Fuel Type_premium unleaded (required)','Engine Fuel Type_flex-fuel (premium unleaded required/E85)'],axis=1)
Y=test['MSRP']

In [1039]:
regressor = DecisionTreeRegressor(random_state = 0) 

regressor.fit(X, Y)

DecisionTreeRegressor(random_state=0)

## Testes:

In [1040]:
tree_pred = regressor.predict(X)

linear_y=results.predict(test.drop(['Year','Number of Doors','highway MPG','Engine Fuel Type_diesel','Engine Fuel Type_electric','Engine Fuel Type_electric','Engine Fuel Type_electric','Engine Fuel Type_flex-fuel (premium unleaded recommended/E85)','Transmission Type_AUTOMATIC','Engine Fuel Type_premium unleaded (required)','Engine Fuel Type_flex-fuel (premium unleaded required/E85)'],axis=1))

tree_pred=pd.Series(tree_pred)

df=pd.DataFrame({'MSRP(test)':test['MSRP'],'Preço(tree)':tree_pred,'Razão(tree)':tree_pred/test['MSRP'],'Preço':linear_y,'Razão(linear)':linear_y/test['MSRP']})

In [1041]:
df=df.dropna()
df.describe()

,MSRP(test),Preço(tree),Razão(tree),Preço,Razão(linear)
count,65.000000,65.000000,65.000000,6.500000e+01,65.000000
mean,53519.969231,61542.717949,8.051828,4.667032e+07,-585.906974
std,87257.370808,76530.744759,19.349862,1.019012e+08,2480.480542
min,2000.000000,2000.000000,0.041537,-1.909128e+07,-7257.560711
25%,7600.000000,25330.000000,0.567502,-6.743511e+06,-1641.955555
50%,38680.000000,38100.000000,1.000000,2.771742e+07,742.042029
75%,50200.000000,63060.000000,5.152140,4.074290e+07,850.941990
max,455500.000000,418950.000000,100.452117,5.211240e+08,1144.070293
